### Импорты

In [20]:
import time
import pickle
import pandas as pd
import torch
from torch import nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

### Загрузка данных

In [21]:
data = pd.read_csv('data/groups_dataset.csv', index_col=[0])
data.head()

,name,link,class
0,Экономика РФ,https://vk.com/ecoruss?trackcode=31eb2085TMD9J...,0
1,"Экономика, которую мы потеряли",https://vk.com/economist?trackcode=cf7670c7dGz...,0
2,"Кафедра ""Экономика"" ТГТУ",https://vk.com/tmbstu.economics?trackcode=4608...,0
3,Коммерсантъ Экономика,https://vk.com/kommersant.economics?trackcode=...,0
4,Economy of Armenia | Экономика Армении,https://vk.com/armeconomy?trackcode=0ccbdc7dwd...,0


### Подготовка данных для обучения

In [22]:
def clear_text(text):
    cleared_text = str()
    for char in text.lower():
        if char.isalpha(): cleared_text += char
        else: cleared_text += ' '
    return cleared_text.strip()

In [24]:
data_iter = list()

for text, cls in zip(data['name'], data['class']):
    try:
        text = clear_text(text)
        data_iter.append((cls, text))
    except: ...

### Перемешивание данных

In [25]:
import random


random.shuffle(data_iter)

### Разбиение на тренировочную и тестировочную выборку

In [26]:
limit = int(len(data_iter) * 0.85)

train_iter, test_iter = data_iter[:limit], data_iter[limit:]

In [27]:
print('Тренировочная выборка: ', len(train_iter))
print('Тестировочную выборка: ', len(test_iter))
print('Всего данных выборка: ', len(data_iter))

Тренировочная выборка:  4590
Тестировочную выборка:  810
Всего данных выборка:  5400


### Кодировка текста с помощью словаря

In [28]:
tokenizer = get_tokenizer('basic_english')

def generate_token(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [29]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

### Разбивка данных на пакеты

In [30]:
import torch
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

### Модель нейронной сети

In [31]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### Функции обучения и тестирования

In [32]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

            
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

### Параметры обучения

In [36]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64

EPOCHS = 20 
LR = 0.1 
BATCH_SIZE = 64 

criterion = torch.nn.CrossEntropyLoss()

In [37]:
total_accu = None

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)

split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

### Обучение нейронной сети

In [38]:
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [39]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.24s | valid accuracy    0.148 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.17s | valid accuracy    0.274 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.18s | valid accuracy    0.391 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.17s | valid accuracy    0.483 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.23s | valid accuracy    0.609 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.22s |

### Тестирование модели

In [41]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() 

In [44]:
acc, total = 0, 0

for cls, text in test_iter:
    if text:
        pred_cls = predict(text, text_pipeline)
        if cls == pred_cls: acc += 1
        total += 1

print('Точность на тестовой выборке: ', round(acc / total, 2), '%')

Точность на тестовой выборке:  0.91 %


### Сохранение модели 

In [47]:
torch.save(model, 'model.pth')